Jonathon Nguyen

ID: 801093003

Homework 1

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
def normalized_data(tensor):
    mean = torch.mean(tensor)
    std = torch.std(tensor)
    new_tensor = (tensor - mean) / std
    
    return new_tensor

In [3]:
model = nn.Sequential(
        nn.Linear(5, 8),
        nn.Tanh(),
        nn.Linear(8, 1))

optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [6]:
NUM_EPOCHS = 200

# Read the data from the provided CSV files
housing = pd.DataFrame(pd.read_csv("Housing.csv"))

# Split the data into the input vars and the prices.
names_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
prices = housing['price']

# Find the length of the column.
num_samples = len(prices)
num_val = int(0.2 * num_samples)

# Generate the random indices with 80% training and 20% Validation
random_indices = torch.randperm(num_samples)
training_indices = random_indices[:-num_val]
validation_indices = random_indices[-num_val:]

# Turn input, prices and params into a tensor
training_vars = []
validation_vars = []
for col in names_vars: 
    # Convert the column into a tensor.
    tensor = torch.tensor(housing[col]).float()
    
    # Split the data set and normalized the data.
    training_tensor = normalized_data(tensor[training_indices])
    validation_tensor = normalized_data(tensor[training_indices])
    
    # Packing the tensor into a list to pass as a param
    training_vars.append(training_tensor)
    validation_vars.append(validation_tensor)
    
training_vars = torch.stack(training_vars[:]).unsqueeze(1)
validation_vars = torch.stack(validation_vars[:])

# Convert the prices to a tensor.
prices = torch.tensor(prices.values).float()

training_vars.shape

torch.Size([5, 1, 436])

In [7]:
temp = model(training_vars)

RuntimeError: size mismatch, m1: [5 x 436], m2: [5 x 8] at ..\aten\src\TH/generic/THTensorMath.cpp:41